In [7]:
#source1: https://nbviewer.org/github/ekontou/CEE-UTM-UTO/tree/master/FrankWolfeforUE-Lecture6.ipynb
#source2: https://nbviewer.org/github/ekontou/CEE-UTM-UTO/tree/master/NetworkX-Basics-v1-EK.ipynb

import pandas as pd
import numpy as np
import networkx as nx
import scipy.optimize as sopt

#define link performance function
def t(x):
    t=np.zeros(len(link))
    t[0]=x[0]+1
    t[1]=x[1]+4
    t[2]=x[2]+1
    return t

#define objective function (sum of integral of t)
def z(x):
    return 0.5*(x[0])**2+x[0]+0.5*(x[1])**2+4*x[1]+0.5*(x[2])**2+x[2]

#define demand
demand = np.array([[1,2,1],[1,3,2]])

#define links
link = np.array([[1,2],[1,3],[2,3]])

#define emission factors
h = np.array([0.01,0.01,0.5])

#implement Dijkstra's Algorithm
def sp(x, t, O, D):
    
    #initialize an empty directed graph object
    G1=nx.DiGraph()

    #add nodes to graph
    i=0
    while(i<len(link)):
        G1.add_node(i+1)
        i+=1
 
    #add link characteristics
    ti=t(x)
    i=0
    while (i<len(link)):
        G1.add_edge(link[i,0],link[i,1], weight = ti[i])
        i+=1
    
    #find shortest path
    sp = nx.dijkstra_path(G1,source = O, target = D)
    return sp

#implement Frank-Wolfe Algorithm
#at Iteration 0
guesses = np.zeros(len(link))
i=0
while (i<len(demand)):
    path = sp(guesses, t, demand[i,0], demand[i,1])
    j=0
    while (j<len(link)):
        k=0
        while (k<len(path)-1):
            if(link[j,0]==path[k] and link[j,1]==path[k+1]):
                guesses[j]+=demand[i,2]
            k+=1
        j+=1
    i+=1 

#at subsequent iterations
def step(s, guesses):

    #find direction y based on min t
    y = np.zeros(len(link))
    i=0
    while (i<len(demand)):
        path = sp(guesses, t, demand[i,0], demand[i,1])
        j=0
        while (j<len(link)):
            k=0
            while (k<len(path)-1):
                if(link[j,0]==path[k] and link[j,1]==path[k+1]):
                    y[j]+=demand[i,2]
                k+=1
            j+=1
        i+=1

    # objective function to minimize
    def func(alpha):
        return z(guesses+alpha*(y-guesses))

    # find alpha that minimizes objective function
    alpha = sopt.minimize(func, 0, bounds=[(0,1)])

    # update x
    last_guess = guesses
    guesses = guesses + alpha.x*(y-guesses)
    return last_guess,guesses

def converge_test(last_guess,next_guess):
    delta = 0.0001
    v1=v2=check=i=0
    while (i<len(guesses)):
        v1 += (next_guess[i]-last_guess[i])**2
        v2 += next_guess[i]
        i+=1
    check=v1**0.5/v2
    return True if (check < delta) else False
          
for s in range(1,10000):
    last_guess,next_guess=step(s,guesses)
    guesses=next_guess
    if converge_test(last_guess,next_guess)==True:
        break

#set print options for numpy arrays
np.set_printoptions(suppress=True, formatter={'float_kind':'{:.0f}'.format})

print('Solution for Slide 14:')
print('x =',guesses)
print('T for OD-pair (1,2) =',np.round(t(guesses)[0],0))
print('T for OD-pair (1,3) =',np.round(t(guesses)[1],0))
print('Total emissions =',np.round(np.sum(guesses*h),2))

Solution for Slide 14:
x = [2 1 1]
T for OD-pair (1,2) = 3.0
T for OD-pair (1,3) = 5.0
Total emissions = 0.53


In [8]:
#amend demand
demand = np.array([[1,2,0.5],[1,3,2]])

for s in range(1,10000):
    last_guess,next_guess=step(s,guesses)
    guesses=next_guess
    if converge_test(last_guess,next_guess)==True:
        break

#set print options for numpy arrays
np.set_printoptions(suppress=True, formatter={'float_kind':'{:.3f}'.format})

print('Solution for Slide 15:')
print('x =',guesses)
print('T for OD-pair (1,2) =',np.round(t(guesses)[0],2))
print('T for OD-pair (1,3) =',np.round(t(guesses)[1],2))
print('Total emissions =',np.round(np.sum(guesses*h),4))

Solution for Slide 15:
x = [1.667 0.833 1.167]
T for OD-pair (1,2) = 2.67
T for OD-pair (1,3) = 4.83
Total emissions = 0.6083
